In [1]:
import os

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import json

In [ ]:
%%time
df = pd.read_csv('../data/scotus_file', usecols = ['author', 'html'])

In [ ]:
print(len(df))
df.dropna(how="any", inplace=True)
print(len(df))

In [ ]:
%%time
def parse_html(raw_html):
    return BeautifulSoup(raw_html, "html.parser").text
df['text'] = df.apply(lambda row: parse_html(row['html']), axis=1)

In [2]:
%%time
df = pd.read_csv('../data/parsed.csv')

CPU times: user 9.6 s, sys: 1.13 s, total: 10.7 s
Wall time: 10.7 s


In [13]:
names = 10000 * [None] # names[i] = (first, middle, last)
PEOPLE_ROOT = "../data/people"
for file_name in os.listdir(PEOPLE_ROOT):
    author_idx = int(file_name.split('.')[0])
    with open(os.path.join(PEOPLE_ROOT, file_name)) as f:
        data = json.load(f)
        names[author_idx] = (data['name_first'], data['name_middle'], data['name_last'])
names[0:5]

[None,
 ('George', '', 'Washington'),
 ('John', '', 'Adams'),
 ('Thomas', '', 'Jefferson'),
 ('James', '', 'Madison')]

In [14]:
def get_idx(author_url): # returns int
    return int(author_url.split('/')[-2])
get_idx(df['author'][df.index[0]])

26

In [20]:
fml = df.apply(lambda row: names[get_idx(row['author'])], axis = 1)
first, middle, last = map(list, zip(*fml))
last[0:5]

['Taft', 'Waite', 'Marshall', 'Stevens', 'Harlan']

In [21]:
df['first'] = first
df['middle'] = middle
df['last'] = last

In [23]:
%%time
df.to_csv('../data/parsedv2.csv')

CPU times: user 28 s, sys: 901 ms, total: 28.9 s
Wall time: 29.5 s
